In [ ]:
import viennaps as ps # Import the ViennaPS Python bindings
ps.setDimension(3) # Set the simulation dimension to 3D

In [ ]:
extent = 30
gridDelta = 0.3

# Create a simulation domain with square XY extent and specified grid spacing
domain = ps.Domain(xExtent=extent, yExtent=extent, gridDelta=gridDelta, boundary=ps.BoundaryType.REFLECTIVE_BOUNDARY)

Add a rectangular [fin](https://viennatools.github.io/ViennaPS/geo/basic/fin.html) structure to the domain:
- finWidth = 6 units
- finHeight = 0 (just a mask feature)
- maskHeight = 10 units (height of the resist/mask layer)

In [ ]:
ps.MakeFin(domain, finWidth=6., finHeight=0., maskHeight=10.0).apply()

In [ ]:
domain.show()

In [ ]:
# Save current geometry as volume mesh
# domain.saveVolumeMesh("emulation_1")
domain.saveSurfaceMesh("emulation_1", addInterfaces=True)

Run a deposition process using [IsotropicProcess](https://viennatools.github.io/ViennaPS/models/prebuilt/isotropic.html).

In [ ]:
# Duplicate the top-level set and assign it to SiO2 material
domain.duplicateTopLevelSet(ps.Material.SiO2) # add new material layer

# Apply an isotropic deposition process (e.g., CVD)
# with a constant rate of 1.0 for a duration of 4.0 time units
isoDepo = ps.IsotropicProcess(rate=1.0)
ps.Process(domain, isoDepo, 4.0).apply()

In [ ]:
# Save the structure after isotropic deposition
domain.saveVolumeMesh("emulation_2")
domain.show()

Etch the structure using a [DirectionalProcess](https://viennatools.github.io/ViennaPS/models/prebuilt/directional.html).

In [ ]:
# Apply directional (anisotropic) etching in Z direction
# - This mimics vertical etching to remove top parts of the conformal layer
# - calculateVisibility=False means no shadowing is considered
matRates = ps.RateSet()
matRates.direction = [0., 0., -1.]
matRates.directionalVelocity = -1.0
matRates.isotropicVelocity = 0.0
matRates.calculateVisibility = False

directionalEtch = ps.DirectionalProcess(matRates)
ps.Process(domain, directionalEtch, 5.0).apply()

In [ ]:
# Save geometry after directional etch
domain.saveVolumeMesh("emulation_3")
domain.show()

In [ ]:
# Remove the original resist/mask material
# This mimics stripping the mask after sidewall transfer
domain.removeMaterial(ps.Material.Mask)
domain.saveVolumeMesh("emulation_4")
domain.show()

In [ ]:
# Final directional etch using the SiO2 sidewalls as a hardmask
# - Directional component: 1.0 in Z
# - Isotropic component: 0.1 to add some tapering
# - Masking is enabled for SiO2
directionalEtch = ps.DirectionalProcess(
    direction=[0., 0., 1.],
    directionalVelocity=1.0,
    isotropicVelocity=0.0,
    maskMaterial=ps.Material.SiO2,
    calculateVisibility=False
)
ps.Process(domain, directionalEtch, 20.0).apply()

In [ ]:
# Save the final result of the emulation
domain.saveVolumeMesh("emulation_5")
domain.show()